In [6]:
# import pandas as pd

# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait
# ActionChains 여러 동작을 체인으로 묶어서 저장하고 실행, 클릭 키보드 입력 등
# from selenium.webdriver.common.action_chains import ActionChains

import copy

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import csv

filename = "real_estate.csv"
f = open(filename, "w", encoding="utf-8-sig", newline="")
writer = csv.writer(f)

attributes = ["매물이름", "월세", "전세", "가격", "소재지", "특징", "공급/전용면적"]
writer.writerow(attributes)

# 부동산 url 접속, 쿼리로 접속하고 싶은데 위치값으로 접속하는거는 아직 모르겠음

url3='https://new.land.naver.com/complexes?ms=37.4853,126.9719,16&a=PRE&e=RETAIL'     # 동작구 사당동 
url2='https://new.land.naver.com/complexes?ms=37.4753,126.9567,16&a=PRE&e=RETAIL'     # 관악구 봉천동
url='https://new.land.naver.com/complexes?ms=37.470101,127.039888,16&a=PRE&e=RETAIL' # 서초구 양재동
url4='https://new.land.naver.com/complexes?ms=37.57,127.0564,16&a=PRE&e=RETAIL'       # 동대문 답시리동
# 0123

driver = webdriver.Chrome()
# driver.maximize_window()
driver.get(url)
driver.implicitly_wait(time_to_wait=20)

# 원룸·투룸 클릭
driver.find_element(By.XPATH, '//*[@id="wrap"]/div[1]/a[3]').click()
driver.implicitly_wait(time_to_wait=10)


# driver.switch_to.frame("searchIframe")
# iframe이 아니어서 밑의 코드로 변경

for i in range(1, 101):
    
    element = driver.find_element(By.XPATH, f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]/div')    
    
    # 네이버에서 보기 있는지 없는지 확인 후 있으면 클릭 아니면 그냥 클릭
    # 처리 안해주면 네이버에서 보기 있는경우 새창에서 열림
    if "네이버에서 보기" in element.text:
        driver.find_element(By.XPATH, f'//*[@id="listContents1"]/div/div/div[1]/div[{i}]/div/div[2]/a').click()
    else:
        element.click()
    driver.implicitly_wait(time_to_wait=2)
    
    # 이거 여기서 안해주면 결과저장시 맨 처음 값 중복출력
    time.sleep(1)
    
    # 상단 이미지 있고없고
    try:
        sale_name = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4 ').text
        sale_price = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3] ').text
        
        # 테이블 첫번째 값이 소재지인지 확인
        if driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/th ').text == '소재지':
            sale_add = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td ').text
            sale_feature = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[2]/td ').text
            sale_area = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[3]/td ').text
        else:
            sale_add = "X"
            sale_feature = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td ').text
            sale_area = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[2]/td ').text
    # except:
    #     pass
    except:
        sale_name = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[2]/h4').text
        sale_price = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]').text
        
        # 테이블 첫번째 값이 소재지인지 확인
        if driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/th').text == "소재지":
            sale_add = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td').text
            sale_feature = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[2]/td').text
            sale_area = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[3]/td ').text
        else:
            sale_add = "X"
            sale_feature = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td').text
            sale_area = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[2]/td ').text
    
    # 전세/월세 구분 및 괄호 안 내용 제거
    index_open = sale_price.find('(')
    
    # '(' 있는지 확인 및 전세월세 구분
    sale_price_month = "X"
    sale_price_year = "X"
    if index_open != -1:
        if sale_price[0:2] == '월세':
            sale_price = sale_price[2:index_open]
            sale_price_month = "O"
        else:
            sale_price = sale_price[2:index_open]
            sale_price_year = "O"
    else:
        if sale_price[0:2] == '월세':
            sale_price = sale_price[2:]
            sale_price_month = "O"
        else:
            sale_price = sale_price[2:]
            sale_price_year = "O"

    # 쉼표 삭제 처리
    if "," in sale_price:
        sale_price = sale_price.replace(",","")
    if "," in sale_feature:
        sale_feature = sale_feature.replace(",","")

    # 쉼표로 나중에 테이블에 데이터를 구분하여 넣을때 오류가 발생하여 ""이거로 각 데이터를 감싸줌
    data_rows = [sale_name,sale_price_month,sale_price_year,
                 sale_price,sale_add,sale_feature,sale_area]
    writer.writerow(data_rows)
    
    print(sale_name + "\t" + sale_price + "\t" +  sale_add + "\t" +  sale_feature + "\t" +  sale_area)

    driver.implicitly_wait(time_to_wait=10)
    driver.execute_script("arguments[0].scrollIntoView(true);", element)

f.close()
driver.close()

The chromedriver version (117.0.5938.149) detected in PATH at c:\Users\AISW-203-115\Desktop\web_crawling\sample_page\chromedriver.exe might not be compatible with the detected chrome version (119.0.6045.105); currently, chromedriver 119.0.6045.105 is recommended for chrome 119.*, so it is advised to delete the driver in PATH and retry


SessionNotCreatedException: Message: session not created: This version of ChromeDriver only supports Chrome version 117
Current browser version is 119.0.6045.105 with binary path C:\Program Files\Google\Chrome\Application\chrome.exe
Stacktrace:
	GetHandleVerifier [0x00007FF6E6F47D12+55474]
	(No symbol) [0x00007FF6E6EB77C2]
	(No symbol) [0x00007FF6E6D6E0EB]
	(No symbol) [0x00007FF6E6DA1022]
	(No symbol) [0x00007FF6E6DA047E]
	(No symbol) [0x00007FF6E6D9B0E7]
	(No symbol) [0x00007FF6E6D986CE]
	(No symbol) [0x00007FF6E6DD713B]
	(No symbol) [0x00007FF6E6DCEF33]
	(No symbol) [0x00007FF6E6DA3D41]
	(No symbol) [0x00007FF6E6DA4F84]
	GetHandleVerifier [0x00007FF6E72AB762+3609346]
	GetHandleVerifier [0x00007FF6E7301A80+3962400]
	GetHandleVerifier [0x00007FF6E72F9F0F+3930799]
	GetHandleVerifier [0x00007FF6E6FE3CA6+694342]
	(No symbol) [0x00007FF6E6EC2218]
	(No symbol) [0x00007FF6E6EBE484]
	(No symbol) [0x00007FF6E6EBE5B2]
	(No symbol) [0x00007FF6E6EAEE13]
	BaseThreadInitThunk [0x00007FF962D5257D+2

정적 페이지로 정보를 디스플레이
조금은 보기좋게 페이지를 디자인
거기 안에는 내가 활용한 도구나 프레임워크들이 제시되어 있어야함

문제사항
1. 매물을 클릭해서 나오는 상세페이지에서
1-1. 상단 이미지가 없어서 div 구조가 바뀌는 경우
    처음에 이미지가 있는지 없는지 if로 나눠서 접근
    -> 메종슈에뜨처럼 위에 이미지는 있는데 소재지가 없는 애들은 제대로 처리되어 나옴
1-2. table에 소재지가 존재하지 않아서 div 구조가 바뀌는 경우
    table에서 소재지 text가 초반에 존재하는지 확인
    -> 에러 제거없이 정상작동


2. 월세 전세 구분
구분해서 엑셀에 표현 완료
근데 이거를 전세나 월세 하나만 값을 가져와서 그 값을 기준으로 데이터를 정리하거나 해야할것같아
2-1 월세/전세 중 하나의 값만 찾아서 출력
    -> 나온 결과값의 비교가 쉬워짐
    -> 월세만 가져온다고 가정할 경우
        - 평수를 계산한다고 하고 근데 이게 의미가 없지 젤 싼방을 찾는게 아니니까

3. 이미지를 가져와야하나 말아야하나
    - 이미지가 없는 데이터가 있어서 어려울듯
    - 이거는 일단 보류

4. 가져온 이 데이터들로 어떤 통계자료나 그래프를 만들어서 의미를 부여 
- 월세가격 평균
- 전세가격 평균
- 지역별로 비교?


프로그램
인터넷 페이지
적당한 구성은 지도 홈페이지에 네이버지도를 구현해놓고 위치를 변경할 수 있게 해 놓고
변경한 위치를 기준으로 밑에 버튼으로 이 위치를 기준으로 매물 찾기를 가능하게 하고싶음
이 버튼을 누르면 지도 밑에 텍스트로 일단 매물들이 나올 수 있게 하고싶고
나올 텍스트는 코드로 구현

지도로 위치를 불러오는게 어려울경우
특정 위치 몇개를 마커로 저장해놓고
그 위치의 주소를 데이터에 입력
주소데이터의 '구'와 '동'을 기준으로 부동산 데이터를 검색 후
밑에 엑셀 형태로 화면에 뿌려줌

지도 하단에 엑셀 형식으로? 일단 데이터 정렬



고민중인 부분

소재지 필요 있나없나, 일단 소재지가 등록 안된 데이터도 많아서 굳이 의미가 있나 싶음
그리고 월세전세 둘중에 하나만 표현해주는게 더 나을 것 같기도하다는 생각이 듭니다.

좋은건 월세 매물보기랑 전세 매물보기를 따로 설정해서 두 버전으로 데이터를 확인할 수 있게 해주면 제일 좋겠지만 지금 하나도 막막한데 일단 하나라도 되게 해야지
그냥 문자중에 쉼표가 있으면 강제로 쉼표를 삭제?
그 후에 처리가 더 쉬울것같긴해

그냥 문자중에 쉼표가 있으면 강제로 쉼표를 삭제?
그 후에 처리가 더 쉬울것같긴해

In [ ]:
try:
    sale_name = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4').text
    sale_price = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3]').text
except:
    sale_name = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[2]/h4').text
    sale_price = driver.find_element(By.XPATH, '//*[@id="ct"]/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]').text

try:
    table_rows = driver.find_elements(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr')
    if table_rows[0].find_element(By.TAG_NAME, 'th').text == '소재지':
        sale_add = table_rows[0].find_element(By.TAG_NAME, 'td').text
        sale_feature = table_rows[1].find_element(By.TAG_NAME, 'td').text
        sale_area = table_rows[2].find_element(By.TAG_NAME, 'td').text
    else:
        sale_add = "X"
        sale_feature = table_rows[0].find_element(By.TAG_NAME, 'td').text
        sale_area = table_rows[1].find_element(By.TAG_NAME, 'td').text
except:
    sale_add = "X"
    sale_feature = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[0]/td').text
    sale_area = driver.find_element(By.XPATH, '//*[@id="detailContents1"]/div[1]/table/tbody/tr[1]/td').text

In [ ]:
# 초기값 설정
sale_price_month = "X"
sale_price_year = "X"

# '월세' 여부 확인
is_monthly_rent = sale_price.startswith('월세')

# 전세/월세 구분 및 괄호 안 내용 제거
index_open = sale_price.find('(')

if index_open != -1:
    sale_price = sale_price[2:index_open]
    sale_price_month = "O" if is_monthly_rent else "X"
else:
    sale_price = sale_price[2:]
    sale_price_year = "O" if not is_monthly_rent else "X"

# 쉼표 제거 처리
sale_price = sale_price.replace(",", "")
sale_feature = sale_feature.replace(",", "")